**ENTRENAMIENTO DE TERSONFLOW PARA DATA DE ELECTROCARDIOGRAMAS**

#NOTACION A USAR *son las variables q no son pertinentes en el estudio
NOTA: Los datos originales poseen una fila que se debe eliminar a mano ya que la misma tiene mas columnas que las otras
1. survival -- the number of months patient survived (has survived, if patient is still alive).  
2. still-alive -- a binary variable.  0=dead at end of survival period, 1 means still alive 
3. age-at-heart-attack -- age in years when heart attack occurred
4. pericardial-effusion -- binary. Pericardial effusion is fluid around the heart.  0=no fluid, 1=fluid
5. fractional-shortening -- a measure of contracility around the heart lower numbers are increasingly abnormal
6. epss -- E-point septal separation, another measure of contractility. Larger numbers are increasingly abnormal.
7. lvdd -- left ventricular end-diastolic dimension.  This is a measure of the size of the heart at end-diastole. Large hearts tend to be sick hearts.
8. wall-motion-score -- a measure of how the segments of the left ventricle are moving
9. wall-motion-index -- equals wall-motion-score divided by number of segments seen.  Usually 12-13 segments are seen in an echocardiogram.  Use this variable INSTEAD of the wall motion score.
10. mult -- a derivate var which can be ignored
11. name -- the name of the patient (I have replaced them with "name")
12. group -- meaningless, ignore it
13. alive-at-1 -- Boolean-valued. Derived from the first two attributes. 0 means patient was either dead after 1 year or had been followed for less than 1 year.  1 means patient was alive at 1 year.

In [1]:
#PAQUETERIAS A USAR 
# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
# Bibliotecas de ayuda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#Biblioteca para datos de entrenamiento
from sklearn.model_selection import train_test_split

**ENTRADA DE LA INFORMACION, PROCESAMIENTO**

In [2]:
# NOTACION DE LAS VARIABLES 
label_total=["survival", "still_alive", "age-at-heart-attack",
       "pericardial-effusion","fractional-shortening", 
       "epss", "lvdd","wall-motion-score","wall-motion-index",
       "mult" , "group", "alive-at-1"];
# ENTRAR LA INFORMACION DESDE UN ARCHIVO INTERNO
data = pd.DataFrame(np.genfromtxt('echocardiogram.data', delimiter=','), columns= label_total)
label_user=["age-at-heart-attack", "pericardial-effusion","fractional-shortening", "epss",
            "lvdd","wall-motion-index", "alive-at-1"];

**PROCESO DE FILTRADO**

In [3]:
## FILTRAR LAS VARIABLES QUE SE UTILIZARAN EN EL ENTRENAMIENTO DE ML
select_data=data[label_user];#DATOS ORIGINALES SIN FILTRAR, COLUMNAS DE INTERES
#ARREGLAR LA COLUMNA DE alive-at-1"];
select_data[["alive-at-1-false"]]=select_data[["alive-at-1"]];
#alive= np.zeros(len(select_data));
for j in range(len(select_data)):
    if   data[label_total[0]][j] > 12:#MAS DE 12 MESES
         select_data["alive-at-1"].values[j] = 1; #CUMPLE EL REQUISITO
         select_data["alive-at-1-false"].values[j] = 0; #CUMPLE EL REQUISITO CONTRARIO
    elif data[label_total[0]][j] == 12 and data[label_total[1]][j] == 1:#12 MESES Y VIVE
         select_data["alive-at-1"].values[j] = 1; #CUMPLE EL REQUISITO
         select_data["alive-at-1-false"].values[j] = 0; #CUMPLE EL REQUISITO CONTRARIO
    elif data[label_total[0]][j] < 12 and data[label_total[1]][j] == 1:# MENOS DE 12 MESES Y VIVE
         select_data["alive-at-1"].values[j] = -1; #NO CUMPLE EL REQUISITO
         select_data["alive-at-1-false"].values[j] = -1; #CUMPLE EL REQUISITO CONTRARIO   
    else:
         select_data["alive-at-1"].values[j] = 0; #CUMPLE EL REQUISITO
         select_data["alive-at-1-false"].values[j] = 1; #CUMPLE EL REQUISITO CONTRARIO
    #from random import random
    #if random()>.5:
    #    select_data["alive-at-1"].values[j] = 0; #ruido
    
##MATRIX LOGICA DE DATOS ELIMINAR LO DESCONOCIDO
for i in range(len(label_user)):
    if i==0:
        log=select_data[label_user[i]] !=-1;#print(i);
    else:
        log= np.logical_and(select_data[label_user[i]] !=-1,log)
select_data=select_data[log]; # filtrar los desconocidos           

##VALORES FINALES QUE SE UTILIZARAN PARA ENTRENARSE CORRECTAMENTE
input_data, input_data_test, output_data, output_data_test = train_test_split(
    select_data[{"age-at-heart-attack", "pericardial-effusion","fractional-shortening", 
                 "epss", "lvdd","wall-motion-index"}], 
    select_data[{"alive-at-1","alive-at-1-false"}], test_size=0.2, shuffle = True);

#output_data[['alive-at-1-false']]=output_data.values# incluir columna logica al output



#input_data=select_data[{"age-at-heart-attack", "pericardial-effusion","fractional-shortening", "epss", "lvdd","wall-motion-index"}];
#output_data=select_data[{"alive-at-1"}];

#GRAFICAR ALGUNOS DATOS
#print(select_data)

C:\Users\Dell\.conda\envs\ML\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [10]:
#output_data[['alive-at-1-false']]=output_data
#print(output_data.values)#np.ones(output_data.size)
print(input_data)

     epss  lvdd  wall-motion-index  pericardial-effusion  \
2     4.0  3.42               1.00                   0.0   
102   0.0  4.55               1.00                   0.0   
9    13.0  4.49               1.19                   0.0   
55   15.6  6.15               1.00                   0.0   
87   19.2  5.25               1.00                   1.0   
..    ...   ...                ...                   ...   
81   16.8  4.16               1.50                   1.0   
115  28.9  6.63               1.95                   1.0   
24    5.9  3.48               1.11                   0.0   
117   9.3  4.79               1.00                   0.0   
57   18.6  4.37               1.37                   0.0   

     fractional-shortening  age-at-heart-attack  
2                     0.26                 55.0  
102                   0.38                 70.0  
9                     0.14                 54.0  
55                    0.29                 66.0  
87                    0.16   

**PARAMETROS INICIALES DE ENTRENAMIENTO**

In [5]:
n_input = np.int64(len(label_user)-1)   # input layer
n_hidden = np.int64(np.round(np.sqrt(len(label_user)))) # 1st hidden layer
n_output = np.int64(2)   # output 

learning_rate = 1e-4;   n_iterations = 100
batch_size = 3;         dropout = 0.5

X = tf.compat.v1.placeholder("float", [None, n_input]);
Y = tf.compat.v1.placeholder("float", [None, n_output])
#keep_prob = tf.placeholder(tf.float32) 

weights = {
    'w1': tf.Variable(tf.random.normal([n_input, n_hidden])),
    'out': tf.Variable(tf.random.normal([n_hidden, n_output]))
    }

biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
#layer_drop = tf.nn.dropout(layer_1, keep_prob)
output_layer = tf.matmul(layer_1, weights['out']) + biases['out']

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
train_step = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

**INICIALIZAR EL ENTRENAMIENTO**

In [6]:
init = tf.compat.v1.global_variables_initializer()

cost_summary = tf.compat.v1.summary.scalar("Cost", cross_entropy)
acc_summary = tf.compat.v1.summary.scalar("Accuracy", accuracy)
all_summary = tf.compat.v1.summary.merge_all()

**ENTRENAMIENTO**

In [12]:
#saver=tf.train.Saver() #SALVAR LA SECCION PARA PODERLA UTILIZAR LUEGO
with tf.Session() as sess:
    writer = tf.summary.FileWriter("Tensorboard", sess.graph)
    #print(sess)
    sess.run(init)
    #saver.save(sess, 'modelo')
    for i in range(n_iterations):
        sess.run(train_step, feed_dict={X: input_data.values, Y: output_data.values})
        #if i % 10 == 0:
        summary_results, loss, acc = sess.run([all_summary, cross_entropy, accuracy], feed_dict={X: input_data.values, Y: output_data.values})
                #    writer.add_summary(summary_results, i)
        print("Iteration", str(i), "\t| Loss =", str(loss), "\t| Accuracy =", str(acc))
    
        if acc==1:
            break
    test_accuracy = sess.run(accuracy, feed_dict={X: input_data_test.values, Y: output_data_test.values})
    print("\nAccuracy on test set:", test_accuracy)
        
    #saver.save(sess, 'modelo') #SALVAR EL MODELO EXTERNAMENTE

Iteration 0 	| Loss = 11.20876 	| Accuracy = 0.9230769
Iteration 1 	| Loss = 11.205221 	| Accuracy = 0.9230769
Iteration 2 	| Loss = 11.201681 	| Accuracy = 0.9230769
Iteration 3 	| Loss = 11.198145 	| Accuracy = 0.9230769
Iteration 4 	| Loss = 11.194607 	| Accuracy = 0.9230769
Iteration 5 	| Loss = 11.191071 	| Accuracy = 0.9230769
Iteration 6 	| Loss = 11.187533 	| Accuracy = 0.9230769
Iteration 7 	| Loss = 11.183999 	| Accuracy = 0.9230769
Iteration 8 	| Loss = 11.180465 	| Accuracy = 0.9230769
Iteration 9 	| Loss = 11.176931 	| Accuracy = 0.9230769
Iteration 10 	| Loss = 11.1734 	| Accuracy = 0.9230769
Iteration 11 	| Loss = 11.169868 	| Accuracy = 0.9230769
Iteration 12 	| Loss = 11.166339 	| Accuracy = 0.9230769
Iteration 13 	| Loss = 11.16281 	| Accuracy = 0.9230769
Iteration 14 	| Loss = 11.159281 	| Accuracy = 0.9230769
Iteration 15 	| Loss = 11.155754 	| Accuracy = 0.9230769
Iteration 16 	| Loss = 11.152229 	| Accuracy = 0.9230769
Iteration 17 	| Loss = 11.148705 	| Accuracy 

**PROBAR EL MODELO CON LA DATA DE MUESTRA**

In [8]:
print(graph)

NameError: name 'graph' is not defined